In [1]:
import skrub
from sklearn.ensemble import HistGradientBoostingClassifier

import sempipes
from sempipes import sem_choose

dataset = skrub.datasets.fetch_credit_fraud()
products = skrub.var("products", dataset.products)
baskets = skrub.var("baskets", dataset.baskets)
baskets = baskets.skb.subsample(n=5000, how="random")

basket_ids = sempipes.as_X(baskets[["ID"]], "Shopping baskets with product transactions")
fraud_flags = sempipes.as_y(baskets["fraud_flag"], "A binary flag indicating a fraudulent shopping basket")

products = products.sem_fillna(
    target_column="make",
    nl_prompt="Infer the manufacturer from relevant product-related attributes like title or description.",
    impute_with_existing_values_only=True,
)

kept_products = products[products["basket_ID"].isin(basket_ids["ID"])]
kept_products = kept_products.sem_gen_features(
    nl_prompt="""
    Generate additional brand- and manufacturer-related product features. Make sure that they can be efficiently computed
    on large datasets, and that they work across a large number of brands and manufacturers. Use your intrinsic knowledge 
    about what products and brands fraudsters focus on to make sure that the new features are helpful for the prediction task 
    at hand.
    """,
    name="brand_features",
    how_many=5,
)

vectorizer = skrub.TableVectorizer()
vectorized_products = kept_products.skb.apply(vectorizer, exclude_cols="basket_ID")

aggregated_products = vectorized_products.groupby("basket_ID").agg("mean").reset_index()
augmented_baskets = basket_ids.merge(aggregated_products, left_on="ID", right_on="basket_ID").drop(
    columns=["ID", "basket_ID"]
)

hgb = HistGradientBoostingClassifier()
fraud_detector = augmented_baskets.skb.apply_with_sem_choose(
    hgb,
    y=fraud_flags,
    choices=sem_choose(
        name="hgb_choices", max_depth="Common range of values for the maximum depth of the learned trees"
    ),
)

2025-11-29 17:26:10,563 - INFO - SEMPIPES> Interactive mode detected, enabling code generation in preview mode.
2025-11-29 17:26:10,840 - INFO - SEMPIPES> Querying 'openai/gpt-4.1' with 2 messages...'
17:26:10 - LiteLLM:INFO: utils.py:3416 - 
LiteLLM completion() model= gpt-4.1; provider = openai
2025-11-29 17:26:10,850 - INFO - SEMPIPES> 
LiteLLM completion() model= gpt-4.1; provider = openai


--- sempipes.sem_fillna('make', 'Infer the manufacturer from relevant product-related attributes like title or description.')


17:26:12 - LiteLLM:INFO: utils.py:1301 - Wrapper: Completed Call, calling success_handler
2025-11-29 17:26:12,915 - INFO - SEMPIPES> Wrapper: Completed Call, calling success_handler


	> Fitting imputation model RandomForestClassifier(random_state=0) on columns ['item', 'model', 'goods_code'] of 108107 rows...
	> Imputing 1273 values...


2025-11-29 17:26:50,748 - INFO - SEMPIPES> Fitting sempipes.sem_gen_features('Generate additional brand- and manu...', 5) on dataframe of shape (9014, 7) in mode 'preview'.
2025-11-29 17:26:50,752 - INFO - SEMPIPES> Querying 'openai/gpt-4.1' with 2 messages...'
17:26:50 - LiteLLM:INFO: utils.py:3416 - 
LiteLLM completion() model= gpt-4.1; provider = openai
2025-11-29 17:26:50,754 - INFO - SEMPIPES> 
LiteLLM completion() model= gpt-4.1; provider = openai
17:26:53 - LiteLLM:INFO: utils.py:1301 - Wrapper: Completed Call, calling success_handler
2025-11-29 17:26:53,539 - INFO - SEMPIPES> Wrapper: Completed Call, calling success_handler
2025-11-29 17:26:53,566 - INFO - SEMPIPES> Computed 5 new feature columns: ['brand_is_apple', 'item_is_computer', 'make_cash_price_mean', 'make_is_retailer', 'make_item_combo_freq'], 


--- sempipes.apply_with_sem_choose(HistGradientBoostingClassifier(), SemChoices(name='hgb_choices', params_and_prompts={'max_depth': 'Common range of values for the maximum depth of the learned trees'}))


2025-11-29 17:26:56,928 - INFO - SEMPIPES> Querying 'openai/gpt-4.1' with 2 messages...'
17:26:56 - LiteLLM:INFO: utils.py:3416 - 
LiteLLM completion() model= gpt-4.1; provider = openai
2025-11-29 17:26:56,931 - INFO - SEMPIPES> 
LiteLLM completion() model= gpt-4.1; provider = openai
17:26:58 - LiteLLM:INFO: utils.py:1301 - Wrapper: Completed Call, calling success_handler
2025-11-29 17:26:58,102 - INFO - SEMPIPES> Wrapper: Completed Call, calling success_handler


	Suggested choices for max_depth: choose_from([7, 3, 5, 10, None], name='__sempipes__...es__max_depth')


In [2]:
fraud_detector

<Call 'store_sem_choices'>
Result (on a subsample):
――――――――――――――――――――――――
0       0
1       0
2       0
3       0
4       0
       ..
4995    0
4996    0
4997    0
4998    0
4999    0
Name: fraud_flag, Length: 5000, dtype: int64

In [3]:
sempipes.inspect_generated_code(fraud_detector, "brand_features")

# (brand_is_apple: Whether the product's make is 'APPLE')
# Input samples: 'make': ['RETAILER', 'APPLE', 'APPLE']
df['brand_is_apple'] = (df['make'] == 'APPLE').astype(int)

# (item_is_computer: Whether the item is a computer-related product)
# Input samples: 'item': ['LIVING DINING FURNITURE', 'COMPUTERS', 'COMPUTERS']
df['item_is_computer'] = df['item'].str.contains('COMPUTER', case=False, na=False).astype(int)

# (make_item_combo_freq: Frequency of (make, item) combination in the dataset)
# Input samples: 'make': ['RETAILER', 'APPLE', 'APPLE'], 'item': ['LIVING DINING FURNITURE', 'COMPUTERS', 'COMPUTERS']
make_item_counts = df.groupby(['make', 'item']).size()
df['make_item_combo_freq'] = df.set_index(['make', 'item']).index.map(make_item_counts).astype(int)

# (make_cash_price_mean: Mean cash price for each make)
# Input samples: 'make': ['RETAILER', 'APPLE', 'APPLE'], 'cash_price': [1899, 1649, 899]
make_cash_price_mean = df.groupby('make')['cash_price'].transform('mean')
df['make_cash_price_mean'] = make_cash_price_mean

# (make_is_retailer: Whether the product's make is 'RETAILER')
# Input samples: 'make': ['RETAILER', 'APPLE', 'APPLE']
df['make_is_retailer'] = (df['make'] == 'RETAILER').astype(int)